<a href="https://colab.research.google.com/github/map72ru/data_mining/blob/main/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606595 sha256=d35b64da7976f5305f7ebf50d3a4dbaf1d7f5d1c3432d1bbd0b8e62743cb469f
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [1]:
from google.colab import drive
drive.mount('/content/hw2')

Mounted at /content/hw2


In [4]:
data = pd.read_csv('hw2/MyDrive/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [6]:
data.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06
mean,1.271904e+03,3.336643e+10,3.639770e+02,2.827247e+06,1.003763e+02,3.100821e+00,3.048227e+03,-5.400708e-01,1.561714e+03,5.268156e+01,-1.638696e-02,-2.897905e-03
std,7.265644e+02,4.284798e+09,1.759385e+02,3.732798e+06,1.152379e+03,4.210229e+00,8.785542e+03,1.245824e+00,4.015691e+02,2.513310e+01,2.168615e-01,3.974618e-02
min,1.000000e+00,2.698485e+10,1.000000e+00,2.567100e+04,0.000000e+00,0.000000e+00,1.000000e+00,-1.300200e+02,0.000000e+00,1.000000e+00,-5.593000e+01,-7.700000e+00
25%,6.550000e+02,3.008714e+10,2.160000e+02,9.169930e+05,1.000000e+00,1.290000e+00,3.300000e+02,-6.900000e-01,1.307000e+03,3.200000e+01,0.000000e+00,0.000000e+00
50%,1.271000e+03,3.241998e+10,3.660000e+02,1.027569e+06,1.000000e+00,2.000000e+00,3.700000e+02,-2.000000e-02,1.614000e+03,5.300000e+01,0.000000e+00,0.000000e+00
75%,1.914000e+03,3.514580e+10,5.150000e+02,1.132178e+06,1.000000e+00,3.490000e+00,4.220000e+02,0.000000e+00,1.844000e+03,7.400000e+01,0.000000e+00,0.000000e+00
max,2.500000e+03,4.165679e+10,6.630000e+02,1.802456e+07,8.963800e+04,8.400000e+02,3.428000e+04,3.990000e+00,2.359000e+03,9.500000e+01,0.000000e+00,0.000000e+00


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [43]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights)
    recs = np.random.choice(items_weights.index, size=n, replace=False, p=items_weights['weight'])

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [47]:
%%time
items = data.groupby(by = "item_id").agg(sales=('sales_value', 'sum'))

##
# Проблема: по некоторым товарным позициям продажи равны нулю. Поэтому явно прологорифмировать нельзя
# поэтому сместим на константу, больше единицы (для того, что бы логарифи не был равен нулю)
items['sales'] = items.sales+1.5
items['weight'] = np.log(items.sales)
##
# Веса должны быть в долях 1 и давать в сумме 1 (по дкументации на np.random.choice)
# Поэтому необходимо нормировать веса
items['weight'] = items.weight.divide(items.weight.sum())

print('sum=', items.weight.sum())
print(items.describe())
items.drop('sales', inplace=True, axis=1)


sum= 1.0
               sales        weight
count   89051.000000  89051.000000
mean       84.958481      0.000011
std      1628.715079      0.000006
min         1.500000      0.000002
25%         5.000000      0.000006
50%        12.280000      0.000010
75%        47.605000      0.000015
max    467995.120000      0.000051
CPU times: user 123 ms, sys: 2.59 ms, total: 125 ms
Wall time: 131 ms


In [46]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[908909, 976996, 48575, 1062095, 932828]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[16733532, 980120, 894103, 870631, 1024508]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
import pandas as pd

result = pd.read_csv('hw2/MyDrive/data/predictions_basic.csv', quotechar='"')
type(result.actual[0])

str

поскольку я так и не разобрался как правильно считать файл (считывается без ошибок, но все значения строки), я написал конвертор 

In [8]:
import re

for column in result.columns:
    if column == 'user_id':
        continue
    col = []
    for row in result[column]:
        row = re.sub('[\[\]]', '', row)
        if column == 'actual':
            sep = ' '
        else:
            sep = ','

        out = []
        for s in row.split(sep):
          s = s.strip()
          if (len(s) > 0):
              out.append(int(s))
        if (len(out) > 0):
            col.append(out)
    result[column] = col

In [14]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):

    recommended_list = recommended_list[:k] 
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [11]:
import numpy as np

In [15]:
p5k = result.apply(lambda row: precision_at_k(row.random_recommendation, row.actual), axis=1)
print('random_recommendation', 100*np.average(p5k))
#
p5k = result.apply(lambda row: precision_at_k(row.popular_recommendation, row.actual), axis=1)
print('popular_recommendation', 100*np.average(p5k))

p5k = result.apply(lambda row: precision_at_k(row.itemitem, row.actual), axis=1)
print('itemitem', 100*np.average(p5k))

p5k = result.apply(lambda row: precision_at_k(row.cosine, row.actual), axis=1)
print('cosine', 100*np.average(p5k))

p5k = result.apply(lambda row: precision_at_k(row.tfidf, row.actual), axis=1)
print('tfidf', 100*np.average(p5k))

p5k = result.apply(lambda row: precision_at_k(row.own_purchases, row.actual), axis=1)
print('own_purchases', 100*np.average(p5k))

random_recommendation 0.029382957884427036
popular_recommendation 15.523996082272282
itemitem 13.692458374143
cosine 13.290891283055831
tfidf 13.898139079333987
own_purchases 17.969311132876264


Лучшим алгоритмом по метрике presision@5 является own_purchases (1 ближайший сосед)

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code